In [ ]:
import pennylane as qml
from pennylane import numpy as np

## V.1.1

In [ ]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev)
def basic_entangling_ansatz(observable,params):
    """Applies an ansatz with basic entanglement.

    Args:
        observable (qml.op): a pennylane operator whose expectation value we want to measure.
        params(np.array): an array with the trainable parameters of the ansatz. They have the shape of `qml.StronglyEntanglingLayers.shape(n_layers=1, n_wires=n_bits)`

    Returns:
        (np.tensor): a numpy tensor of 1 element corresponding to the expectation value of the given observable.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    qml.BasicEntanglerLayers(params, wires=range(n_bits))
    return qml.expval(observable)


@qml.qnode(dev)
def strongly_entangling_ansatz(observable,params):
    """Applies an ansatz with moderate entanglement.

    Args:
        observable (qml.op): a pennylane operator whose expectation value we want to measure.
        params(np.array): an array with the trainable parameters of the ansatz. They have the shape of `qml.StronglyEntanglingLayers.shape(n_layers=1, n_wires=n_bits)`

    Returns:
        (np.tensor): a numpy tensor of 1 element corresponding to the expectation value of the given observable.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    qml.StronglyEntanglingLayers(params, wires=range(n_bits), ranges=(3,))
    return qml.expval(observable)


## V.1.2a

In [ ]:
dev = qml.device('default.qubit', wires=1)
# Define the quantum circuit with a parameterized RY gate
@qml.qnode(dev)
def circuit_to_differentiate(theta):
    """Quantum circuit we want to differentiate.

    Args:
        theta (float): parameter of the circuit with respect to which we differentiate.

    Returns:
        (np.tensor): a numpy tensor of 1 element corresponding to the expectation value of Z
    """
    ##################
    # YOUR CODE HERE #
    ##################
    qml.RY(theta, wires=0)
    return qml.expval(qml.Z(0))

# Define the parameter-shift rule function
def parameter_shift_rule(theta):
    """Function that applies the parameter shift rule to `circuit_to_differentiate` with respect to the parameter theta.

    Args:
        theta (float): parameter of the circuit with respect to which we differentiate.

    Returns:
        (np.tensor): a numpy tensor of 1 element corresponding to the derivative of the circuit with respect to theta.
    """    
    ##################
    # YOUR CODE HERE #
    ##################
    return (circuit_to_differentiate(theta + np.pi / 2) - circuit_to_differentiate(theta - np.pi / 2)) / 2


## V.1.2b

In [ ]:
# Define the built-in parameter-shift rule function
def parameter_shift_rule_built_in(circuit, theta):
    """Function that applies the PennyLane built-in parameter shift rule to a specific circuit with respect to the parameter theta.

    Args:
        circuit (qml.QNode): quantum circuit we want to differentiate.
        theta (float): parameter of the circuit with respect to which we differentiate.

    Returns:
        (np.tensor): a numpy tensor of 1 element corresponding to the derivative of the circuit with respect to theta.
    """  
    ##################
    # YOUR CODE HERE #
    ##################
    return qml.gradients.param_shift(circuit)(theta)

# Define the built-in classical_jacobian function
def jacobian_built_in(circuit, theta):
    """Function that applies the PennyLane built-in jacobian method to a specific circuit with respect to the parameter theta.

    Args:
        circuit (qml.QNode): quantum circuit we want to differentiate.
        theta (float): parameter of the circuit with respect to which we differentiate.

    Returns:
        (np.tensor): a numpy tensor of 1 element corresponding to the derivative of the circuit with respect to theta.
    """  
    ##################
    # YOUR CODE HERE #
    ##################
    return qml.jacobian(circuit)(theta) 


## V.1.3a

In [ ]:
def gradient_descent_optimization(quantum_circuit, initial_theta, learning_rate, max_iterations):
    """
    Performs Gradient Descent optimization to find the optimal parameter theta
    for a quantum circuit to minimize its output expectation value.

    Args:
        quantum_circuit (qml.QNode): A quantum circuit that depends of a parameter.
        initial_theta (np.array): An array with the initial value of the trainable parameter theta.
        learning_rate (float): Learning rate for the gradient descent update.
        max_iterations (int): Maximum number of iterations for the optimization.

    Returns:
        (np.array): A numpy array of 1 element corresponding to the optimized parameter theta.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    jacobian = qml.gradients.param_shift(quantum_circuit)
    theta = initial_theta
    for _ in range(max_iterations):
        gradient = jacobian(theta)
        theta = theta - learning_rate * gradient
    return theta

optimized_theta=gradient_descent_optimization(circuit, np.array(0.1,requires_grad=True), 0.3, 50)
print(f"Optimized theta using Gradient Descent: {optimized_theta}")
print(f"Expectation value of quantum circuit at optimized theta: {circuit(optimized_theta)}")

## V.1.3b

In [ ]:
def gradient_descent_optimization_built_in(quantum_circuit,initial_theta, learning_rate, max_iterations):
    """
    Implements Gradient Descent optimization method of PennyLane to find the optimal parameter theta
    for a quantum circuit to minimize its output expectation value.

    Args:
        quantum_circuit (qml.QNode): A quantum circuit that depends of a parameter.
        initial_theta (np.array): An array with the initial value of the trainable parameter theta.
        learning_rate (float): Learning rate for the gradient descent update.
        max_iterations (int): Maximum number of iterations for the optimization.

    Returns:
        (np.array): A numpy array of 1 element corresponding to the optimized parameter theta.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    optimizer = qml.GradientDescentOptimizer(learning_rate)
    theta = initial_theta
    for _ in range(max_iterations):
        theta = optimizer.step(quantum_circuit, theta)
    return theta

optimized_theta=gradient_descent_optimization_built_in(circuit,np.array(0.1,requires_grad=True), 0.3, 50)
print(f"Optimized theta using built-in Gradient Descent: {optimized_theta}")
print(f"Expectation value of quantum circuit at optimized theta: {circuit(optimized_theta)}")



## V.1.4

In [ ]:
def cost_function(observable,params):
    """Computes the cost function we want to minimize.

    Args:
        observable (qml.Hamiltonian): a pennylane Hamiltonian whose expectation value we want to measure.
        params(np.array): an array with the trainable parameters of the ansatz.

    Returns:
        (np.tensor): a numpy tensor of 1 element corresponding to the cost function value
    """
    ##################
    # YOUR CODE HERE #
    ##################
    return strongly_entangling_ansatz(observable, params)

def optimizer(observable,params):
    """Updates the parameters to minimize the cost function value.

    Args:
        observable (qml.Hamiltonian): a pennylane Hamiltonian whose expectation value we want to measure.
        params(np.array): an array with the trainable parameters of the ansatz.

    Returns:
        (np.array): an array with the optimized trainable parameters.
    """
    # def cost_fn(weights):
    #     return cost_function(observable, weights)

    # max_steps = 100
    # opt = qml.AdamOptimizer(0.1)  

    # for _ in range(max_steps):
    #     # update the weights by one optimizer step
    #     params = opt.step(cost_fn, params)
    # return params
    ##################
    # YOUR CODE HERE #
    ##################
    def cf(params):
        return cost_function(observable, params)
    optimizer = qml.AdamOptimizer()
    prev, params = params, optimizer.step(cf, params)
    while not np.allclose(prev, params):
        prev, params = params, optimizer.step(cf, params)
    return params
